In [2]:
import pandas as pd
import numpy as np

In [3]:
# Leo el dataframe
path = "yellow_taxis_final/yellow_taxis_final_yellow_taxis_transformado_merged.parquet"
df = pd.read_parquet(path)

In [5]:
# Convierto datos numéricos a 32 bit para optimizar procesamiento

df['PULocationID'] = df['PULocationID'].astype('int32')
df['DOLocationID'] = df['DOLocationID'].astype('int32')
df['trip_miles'] = df['trip_miles'].astype('float32')
df['trip_time'] = df['trip_time'].astype('int32')
df['tips'] = df['tips'].astype('float32')
df['passenger_count'] = df['passenger_count'].astype('int32')
df['payment_type'] = df['payment_type'].astype('int32')
df['total_fare'] = df['total_fare'].astype('float32')

In [6]:
# Filtramos outliers

df = df[(df['total_fare'] >= 3) & (df['total_fare'] < 200)]
df = df[(df['trip_miles'] >= 0.2) & (df['trip_miles'] < 50)]
df = df[(df['trip_time'] >= 100) & (df['trip_time'] < 5000)]
df.reset_index(inplace=True, drop=True)



In [8]:
# Creamos 2 nuevas columnas necesarias para graficar KPI´s en Looker

# Columna de turnos: Genera tres turnos
# Turno 1 : De 4 AM a 12 PM
# Turno 2 : De 12 PM a 8 PM
# Turno 3 : De 8 PM a 4 AM

# Define la función para asignar los valores
def asignar_banda(hora):
    if 4 <= hora.hour < 12:
        return 1
    elif 12 <= hora.hour < 20:
        return 2
    else:  # 20 <= hora.hour < 4
        return 3

# Crea la nueva columna de turno
df['ride'] = df['PUTime'].apply(asignar_banda)
df['ride'] = df['ride'].astype('int32')


In [9]:
# Columna de tarifa cobrada por milla recorrida

df['fare_mile'] = df['total_fare'] / df['trip_miles']
df['fare_mile'] = df['fare_mile'].astype('float32')


In [12]:
# Guardamos el archivo localmente para ser subido a Storage

df.to_parquet("yellow_taxis_final/tabla_yellowtaxis_dashboard.parquet")